# Recommending with surprise

## Preliminaries

### imports

In [1]:
import pickle
import random
from time import time
from typing import Tuple, List

In [2]:
import numpy as np
import pandas as pd

In [3]:
import surprise

In [4]:
from surprise.reader import Reader
from surprise import Trainset
from surprise import Dataset
from surprise.model_selection import PredefinedKFold

### Fix experiment stochasity

In [5]:
my_seed = 0
random.seed(my_seed)
np.random.seed(my_seed)

## Load surprise dataset

In [6]:
reader = Reader(
    line_format='user item rating',
    sep=',',
    rating_scale=(1, 5),
)   

In [7]:
folded_trainval_dataset = Dataset.load_from_folds(
    folds_files=(
        ('data/train_for_week_4.csv', 'data/val_for_week_4.csv'),
        ('data/train_for_week_5.csv', 'data/val_for_week_5.csv'),
        ('data/train_for_week_6.csv', 'data/val_for_week_6.csv'),
    ),
    reader=reader,
)

In [8]:
folded_trainval_dataset

In [9]:
unfolded_trainval_dataset = Dataset.load_from_file(
    file_path='data/trainval_weeks_1_to_7.csv',
    reader=reader,
)

In [10]:
unfolded_trainval_dataset

In [11]:
full_trainset_unfolded_trainval_dataset = unfolded_trainval_dataset.build_full_trainset()
full_testset_unfolded_trainval_dataset = full_trainset_unfolded_trainval_dataset.build_testset()

In [12]:
test_dataset = Dataset.load_from_file(
    file_path='data/test_weeks_8_to_9.csv',
    reader=reader,
)
test_dataset

In [13]:
testset_for_test_dataset = test_dataset.build_full_trainset().build_testset()

## Models

In [14]:
from surprise import (
    SVD,
    NormalPredictor,
)
from surprise.model_selection import GridSearchCV

In [15]:
from surprise import accuracy

## Prototyping CV code

### Baseline 1 - NormalPredictor

In [7]:
pkf = PredefinedKFold()

In [8]:
algo = NormalPredictor()

rmse_vals = []
mse_vals = []
mae_vals = []
for trainset, testset in pkf.split(dataset):

    # train and test algorithm.
    algo.fit(trainset)
    predictions = algo.test(testset)

    # Compute and print Root Mean Squared Error
    rmse_vals.append(accuracy.rmse(predictions, verbose=True))
    mse_vals.append(accuracy.mse(predictions, verbose=True))
    mae_vals.append(accuracy.mae(predictions, verbose=True))

RMSE: 1.8658
MSE: 3.4813
MAE:  1.4218
RMSE: 1.8662
MSE: 3.4828
MAE:  1.4227
RMSE: 1.8697
MSE: 3.4956
MAE:  1.4261


In [9]:
rmse_vals

[1.8658342471592386, 1.8662267000996486, 1.8696625436523928]

### Prototype CV

In [26]:
pkf = PredefinedKFold()

param_grid = {'n_epochs': [5, 10, 20], 'lr_all': [0.02, 0.005, 0.01]}
grid_search = GridSearchCV(
    SVD, param_grid, measures=['rmse'], cv=pkf,
    n_jobs=-1,
    # pre_dispatch=4,
    joblib_verbose=51,
)
grid_search.fit(trainval_dataset)

algo = grid_search.best_estimator['rmse']

# # retrain on the whole set A
# trainset = data.build_full_trainset()
# algo.fit(trainset)

# # Compute biased accuracy on A
# predictions = algo.test(trainset.build_testset())
# print('Biased accuracy on A,', end='   ')
# accuracy.rmse(predictions)

# # Compute unbiased accuracy on B
# testset = data.construct_testset(B_raw_ratings)  # testset is now the set B
# predictions = algo.test(testset)
# print('Unbiased accuracy on B,', end=' ')
# accuracy.rmse(predictions)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done   4 out of  18 | elapsed:  1.4min remaining:  4.9min
[Parallel(n_jobs=-1)]: Done   5 out of  18 | elapsed:  1.5min remaining:  3.8min
[Parallel(n_jobs=-1)]: Done   6 out of  18 | elapsed:  1.5min remaining:  3.1min
[Parallel(n_jobs=-1)]: Done   7 out of  18 | elapsed:  1.6min remaining:  2.5min
[Parallel(n_jobs=-1)]: Done   8 out of  18 | elapsed:  1.6min remaining:  2.1min
[Parallel(n_jobs=-1)]: Done   9 out of  18 | elapsed:  2.6min remaining:  2.6min
[Parallel(n_jobs=-1)]: Done  10 out of  18 | elapsed:  7.3min remaining:  5.8min
[Parallel(n_jobs=-1)]: Done  11 out of  18 | elapsed:  7.7min remaining:  4.9min
[Parallel(n_jobs=-1)]: Done  12 out of  18 | elapsed:  7.8min remaining:  3.9min
[Parallel

In [29]:
res_df = pd.DataFrame(grid_search.cv_results)
res_df

,split0_test_rmse,split1_test_rmse,split2_test_rmse,mean_test_rmse,std_test_rmse,rank_test_rmse,mean_fit_time,std_fit_time,mean_test_time,std_test_time,params,param_n_epochs,param_lr_all
0,1.369322,1.369509,1.369781,1.369537,0.000188,1,51.999236,3.109121,15.250752,0.516919,"{'n_epochs': 10, 'lr_all': 0.005}",10,0.005
1,1.376567,1.377377,1.376774,1.376906,0.000344,2,58.687739,0.664675,16.014069,0.163058,"{'n_epochs': 10, 'lr_all': 0.01}",10,0.010
2,1.407132,1.406298,1.406425,1.406618,0.000367,6,62.739012,4.752382,16.443484,0.721562,"{'n_epochs': 10, 'lr_all': 0.1}",10,0.100
3,1.386151,1.385178,1.385121,1.385483,0.000472,3,342.950832,6.344473,27.421346,3.329025,"{'n_epochs': 50, 'lr_all': 0.005}",50,0.005
4,1.386127,1.385602,1.385951,1.385893,0.000218,4,347.730069,6.009606,27.808594,0.849362,"{'n_epochs': 50, 'lr_all': 0.01}",50,0.010
5,1.390280,1.391332,1.392239,1.391283,0.000800,5,291.377803,71.154073,12.476135,7.593410,"{'n_epochs': 50, 'lr_all': 0.1}",50,0.100


In [32]:
best_res = res_df[res_df['rank_test_rmse'] == 1].iloc[0]
best_res

split0_test_rmse                             1.369322
split1_test_rmse                             1.369509
split2_test_rmse                             1.369781
mean_test_rmse                               1.369537
std_test_rmse                                0.000188
rank_test_rmse                                      1
mean_fit_time                               51.999236
std_fit_time                                 3.109121
mean_test_time                              15.250752
std_test_time                                0.516919
params              {'n_epochs': 10, 'lr_all': 0.005}
param_n_epochs                                     10
param_lr_all                                    0.005
Name: 0, dtype: object

In [39]:
best_params = best_res['params']
best_params

{'n_epochs': 10, 'lr_all': 0.005}

In [44]:
# retrain on the whole trainval set
algo.fit(full_trainset_unfolded_trainval_dataset)

#### Metrics on train set

In [58]:
predictions = algo.test(full_testset_unfolded_trainval_dataset)
print(f"Train set RMSE: {accuracy.rmse(predictions)}")
print(f"Train set MSE: {accuracy.mse(predictions)}")
print(f"Train set MAE: {accuracy.mae(predictions)}")

RMSE: 1.0267
Train set RMSE: 1.0266569410037207
MSE: 1.0540
Train set MSE: 1.0540244745111174
MAE:  0.7787
Train set MAE: 0.7786977912006923


#### Metrics on test set

In [56]:
predictions = algo.test(testset_for_test_dataset)
print(f"Test set RMSE: {accuracy.rmse(predictions)}")
print(f"Test set MSE: {accuracy.mse(predictions)}")
print(f"Test set MAE: {accuracy.mae(predictions)}")

RMSE: 1.3423
Test set RMSE: 1.3422636168111464
MSE: 1.8017
Test set MSE: 1.80167161701494
MAE:  1.0708
Test set MAE: 1.0707513252940415


In [59]:
algo

In [70]:
type(predictions[0])

surprise.prediction_algorithms.predictions.Prediction

## CV Code

In [16]:
def _time_print(start):
    seconds = time() - start
    m, s = divmod(seconds, 60)
    h, m = divmod(m, 60)
    h, m, s = int(h), int(m),int(s)
    print(f'Done. Took {h:0>2}:{m:0>2}:{s:0>2} [hour:min:sec]')

In [17]:
class Res:
    GSCV = 'GridSearchCV'
    BEST_MODEL = 'BestModel'
    TEST_PRED = 'TestSetPredictions'
    TEST_RMSE = 'TestRMSE'
    TEST_MAE = 'TestMAE'

In [27]:
def run_grid_search_cv_for_model_and_save_results(
    model_class: surprise.AlgoBase,
    param_grid: dict,
) -> dict:
    """Run grid search CV on a given model on parameter grid.
    
    Parameters
    ----------
    model_class : object
        The class of the model to grid search parameters for.
        A subclass of surprise.AlgoBase.
    param_grid : dict
        A parameter grid, mapping parameter names to list of
        possible values.
    
    Returns
    -------
    results : dict
        Mapping the following result objects:
        'GridSearchCV': surprise.model_selection.GridSearchCV
            The fitted GridSearchCV objects. See grid_search.cv_results.
        'BestModel': surprise.AlgoBase
            An instance of the model class initialized with the best
            parameters and fitted on the entire train-val set.
        'TestSetPredictions': List[surprise.prediction_algorithms.predictions.Prediction]
            The predictions of the aforementioned best model on the (holdout)
            test set, as a list of surprise.Prediction objects.
        'TestRMSE': float
            RMSE on the test set.
        'TestMAE': float
            MAE on the test set.
    """
    
    pkf = PredefinedKFold()

    print(f"Starting grid search for model {model_class} and parameter grid:")
    print(param_grid)
    # we have 3 folds so multiply by 3
    expected_n_jobs = int(3 * np.prod([len(v) for v in param_grid.values()]))
    print(f"\nExpecting to run {expected_n_jobs:0>0} jobs...")
    
    
    start = time()
    grid_search = GridSearchCV(
        algo_class=model_class,
        param_grid=param_grid,
        measures=['rmse', 'mae'],
        cv=pkf,
        n_jobs=-1,
        # pre_dispatch=4,
        joblib_verbose=51,
    )
    grid_search.fit(folded_trainval_dataset)
    _time_print(start)

    best_algo = grid_search.best_estimator['rmse']
    res_df = pd.DataFrame(grid_search.cv_results)
    best_res = res_df[res_df['rank_test_rmse'] == 1].iloc[0]
    best_params = best_res['params']
    print("\nBest results by RMSE:")
    print(best_res)
    print("\nBest params:")
    print(best_params)
    
    print("\nFitting on the entire train-val set...")
    start = time()
    best_algo.fit(full_trainset_unfolded_trainval_dataset)
    _time_print(start)
    
    print("\nPredicting on the train-val set...")
    start = time()
    predictions = best_algo.test(full_testset_unfolded_trainval_dataset)
    _time_print(start)
    print(f"Train set RMSE: {accuracy.rmse(predictions)}")
    print(f"Train set MSE: {accuracy.mse(predictions)}")
    print(f"Train set MAE: {accuracy.mae(predictions)}")
    
    print("\nPredicting on the test set...")
    start = time()
    predictions = best_algo.test(testset_for_test_dataset)
    _time_print(start)
    print(f"Test set RMSE: {accuracy.rmse(predictions)}")
    print(f"Test set MSE: {accuracy.mse(predictions)}")
    print(f"Test set MAE: {accuracy.mae(predictions)}")
    
    print("\nDONE! Grid search terminating...")
    return {
        Res.GSCV: grid_search,
        Res.BEST_MODEL: best_algo,
        Res.TEST_PRED: predictions,
        Res.TEST_RMSE: accuracy.rmse(predictions),
        Res.TEST_MAE: accuracy.mae(predictions),
    }

### Baseline 1 - NormalPredictor

In [28]:
MODEL_CLASS = NormalPredictor
MODEL_CLASS

surprise.prediction_algorithms.random_pred.NormalPredictor

In [29]:
model_res = run_grid_search_cv_for_model_and_save_results(
    model_class=MODEL_CLASS,
    param_grid={},
)

Starting grid search for model <class 'surprise.prediction_algorithms.random_pred.NormalPredictor'> and parameter grid:
{}

Expecting to run 3 jobs...
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    9.8s
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:   10.6s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:   10.6s finished
Done. Took 00:00:23 [hour:min:sec]

Best results by RMSE:
split0_test_rmse    1.872231
split1_test_rmse    1.872019
split2_test_rmse    1.877698
mean_test_rmse      1.873983
std_test_rmse       0.002629
rank_test_rmse             1
split0_test_mae     1.427481
split1_test_mae     1.425985
split2_test_mae     1.432183
mean_test_mae        1.42855
std_test_mae        0.002641
rank_test_mae              1
mean_fit_time       0.974637
std_fit_time        0.185207
mean_test_time      3.804179
std_test_time       1.490243
params                    {}
Name: 0,

In [30]:
model_res.keys()

dict_keys(['GridSearchCV', 'BestModel', 'TestSetPredictions', 'TestRMSE', 'TestMAE'])

In [31]:
strkls = str(MODEL_CLASS)
strkls = strkls[strkls.rfind('.')+1:-2]
print("Pickling model results for:")
print(strkls)

with open(f'results/{strkls}_results.pkl', 'wb+') as f:
    pickle.dump(model_res, f)

Pickling model results for:
NormalPredictor


### Baseline 2 - BaselineOnly

In [32]:
from surprise import BaselineOnly

In [33]:
MODEL_CLASS = BaselineOnly
MODEL_CLASS

surprise.prediction_algorithms.baseline_only.BaselineOnly

In [34]:
baseline_res = run_grid_search_cv_for_model_and_save_results(
    model_class=MODEL_CLASS,
    param_grid={'verbose': [True, False]},
)

Starting grid search for model <class 'surprise.prediction_algorithms.baseline_only.BaselineOnly'> and parameter grid:
{'verbose': [True, False]}

Expecting to run 6 jobs...
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   10.4s
[Parallel(n_jobs=-1)]: Done   2 out of   6 | elapsed:   10.4s remaining:   20.8s
[Parallel(n_jobs=-1)]: Done   3 out of   6 | elapsed:   10.4s remaining:   10.4s
[Parallel(n_jobs=-1)]: Done   4 out of   6 | elapsed:   14.6s remaining:    7.3s
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:   15.3s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:   15.6s finished
Done. Took 00:00:26 [hour:min:sec]

Best results by RMSE:
split0_test_rmse              1.37417
split1_test_rmse             1.366091
split2_test_rmse             1.363647
mean_test_rmse               1.367969
std_test_rmse                0.004496
rank_test_rmse                      1
split0_

In [35]:
baseline_res.keys()

dict_keys(['GridSearchCV', 'BestModel', 'TestSetPredictions', 'TestRMSE', 'TestMAE'])

In [36]:
strkls = str(MODEL_CLASS)
strkls = strkls[strkls.rfind('.')+1:-2]
print("Pickling model results for:")
print(strkls)

with open(f'results/{strkls}_results.pkl', 'wb+') as f:
    pickle.dump(baseline_res, f)

Pickling model results for:
BaselineOnly


### SVD

In [38]:
MODEL_CLASS = SVD
MODEL_CLASS

surprise.prediction_algorithms.matrix_factorization.SVD

In [39]:
svd_res = run_grid_search_cv_for_model_and_save_results(
    model_class=MODEL_CLASS,
    # param_grid={
    #     'n_factors': [10],
    #     'n_epochs': [5], 
    #     'lr_all': [0.02],
    #     'reg_all': [0.01, 0.02],
    # },
    param_grid={
        'n_factors': [10, 50, 100],
        'n_epochs': [5, 10, 20], 
        'lr_all': [0.02, 0.005, 0.01],
        'reg_all': [0.05],
    },
)

Starting grid search for model <class 'surprise.prediction_algorithms.matrix_factorization.SVD'> and parameter grid:
{'n_factors': [10, 50, 100], 'n_epochs': [5, 10, 20], 'lr_all': [0.02, 0.005, 0.01], 'reg_all': [0.05]}

Expecting to run 81 jobs...
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   17.8s
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   21.2s
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:   25.5s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:   36.4s
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:   40.0s
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:   43.0s
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:   47.3s
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:   50.7s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   50.7s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done  11 tasks      | elapsed: 

/Users/shaypalachy/.pyenv/versions/3.10.2/envs/py3/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py:702: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[Parallel(n_jobs=-1)]: Done  57 tasks      | elapsed: 10.9min
[Parallel(n_jobs=-1)]: Done  58 tasks      | elapsed: 11.1min
[Parallel(n_jobs=-1)]: Done  59 tasks      | elapsed: 11.2min
[Parallel(n_jobs=-1)]: Done  60 tasks      | elapsed: 11.6min
[Parallel(n_jobs=-1)]: Done  61 tasks      | elapsed: 11.7min
[Parallel(n_jobs=-1)]: Done  62 tasks      | elapsed: 11.9min
[Parallel(n_jobs=-1)]: Done  63 tasks      | elapsed: 12.0min
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed: 12.8min
[Parallel(n_jobs=-1)]: Done  65 tasks      | elapsed: 13.1min
[Parallel(n_jobs=-1)]: Done  66 tasks      | elapsed: 13.2min
[Parallel(n_jobs=-1)]: Done  68 out of  81 | elapsed: 13.9min remaining:  2.7min
[Parallel(n_jobs=-1)]: Done  70 out of  81 | elapsed: 14.4min remaining:  2.3min
[Parallel(n_jobs=-1)]: Done  72 out of  81 | elapsed: 15.5min remaining:  1.9min
[Parallel(n_jobs=-1)]: Done  74 out of  81 | elapsed: 17.6min remaining:  1.7min
[Parallel(n_jobs=-1)]: Done  76 out of  81 | elapsed: 18

In [40]:
svd_res.keys()

dict_keys(['GridSearchCV', 'BestModel', 'TestSetPredictions', 'TestRMSE', 'TestMAE'])

In [41]:
strkls = str(MODEL_CLASS)
strkls = strkls[strkls.rfind('.')+1:-2]
print("Pickling model results for:")
print(strkls)

with open(f'results/{strkls}_results.pkl', 'wb+') as f:
    pickle.dump(svd_res, f)

Pickling model results for:
SVD
Estimating biases using als...


### SVD++

In [42]:
from surprise import SVDpp

In [43]:
MODEL_CLASS = SVDpp
MODEL_CLASS

surprise.prediction_algorithms.matrix_factorization.SVDpp

In [ ]:
svdpp_res = run_grid_search_cv_for_model_and_save_results(
    model_class=MODEL_CLASS,
    param_grid={
        'n_factors': [10, 75, 150],
        'n_epochs': [5, 20, 40], 
        'lr_all': [0.02, 0.005, 0.01],
        'reg_all': [0.05],
    },
)

Starting grid search for model <class 'surprise.prediction_algorithms.matrix_factorization.SVDpp'> and parameter grid:
{'n_factors': [10, 75, 150], 'n_epochs': [5, 20, 40], 'lr_all': [0.02, 0.005, 0.01], 'reg_all': [0.05]}

Expecting to run 81 jobs...
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  6.5min
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  6.8min
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:  7.0min
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:  8.6min
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  8.8min
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:  9.1min
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed: 10.4min
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed: 10.9min
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed: 18.7min
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed: 35.3min
[Parallel(n_jobs=-1)]: Done  11 tasks      | elapsed

In [ ]:
svdpp_res.keys()

In [ ]:
strkls = str(MODEL_CLASS)
strkls = strkls[strkls.rfind('.')+1:-2]
print("Pickling model results for:")
print(strkls)

with open(f'results/{strkls}_results.pkl', 'wb+') as f:
    pickle.dump(svdpp_res, f)